#### <font color="green">Embedding</font>
- Transform the small parts of text in numbers (vectors) that are easily stored and searched by vector databases.

In [ ]:
# !pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

#### <font color="green">Install LangChain</font>

In [ ]:
# !pip install Langchain

#### <font color="green">Connect with an LLM</font>

In [2]:
# !pip install langchain-openai
# !pip install langchain-groq

###### <font color="blue">NOTE:</font> Since right now is the best LLM in the market, We will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral Models.

In [3]:
from langchain_groq import ChatGroq

chatModel = ChatGroq(model="llama3-70b-8192")

#### <font color="green">Reminder: steps of the RAG process.</font>

- When you load a document, you end up with strings. Sometimes the strings will be too large to fit into the context window. In those occassions. We will use the RAG technique:
  - **Split** document in **small chunks.**
  - **Transform** text chunks in **numeric chunks** (embeddings).
  - Load **embedding to a vector** database (aka vector store).
  - Load question and **retrieve** the most relevant embeddings to respond it.
  - Sent the **embeddings to the LLM** to format the response properly.

#### <font color="green">Splitters: Divide the loaded document in small chunks of text</font>

- Also called "Document Tranformers".
- See the document page <a href="https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/">here.</a>
- See the list of built-in splitters <a href="https://python.langchain.com/v0.1/docs/integrations/document_transformers/">here.</a>

<font color="blue">Simple splitting by character</font>

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/ageye-meta-data.txt")

loaded_data = loader.load()

In [5]:
loaded_data

[Document(metadata={'source': './data/ageye-meta-data.txt'}, page_content='What is our goal?\n\nEnsure that fresh, nutritious, and\nsustainable food is accessible to all.\n\nAs urbanization increases and space becomes a premium, indoor hydroponic farming is not just a choice; itâ€™s a necessity. And for such an important mission, of building an equitable food system, we need solutions that are smart, intuitive, and efficient.\n\nA Global Problem\n\nFacing the Future of Food: A Critical Juncture\n\nDemand Skyrockets, Supply Dwindles\n\nBy the year 2050, our world will be home to nearly 10 billion people, all needing sustenance. Yet, the capacity of our planet to provide is under unprecedented pressure. Valuable agricultural land is vanishing, swallowed by urban expansion, challenged by changing climates, and degraded by practices that fail to stand the test of sustainability.\n\nThe Cost of Conventional Farming\n\nThe way we grow food now demands too much water and leans heavily on chem

In [6]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
text = text_splitter.create_documents([loaded_data[0].page_content])

In [8]:
len(text)

25

In [9]:
text[0]

Document(page_content='What is our goal?\n\nEnsure that fresh, nutritious, and\nsustainable food is accessible to all.\n\nAs urbanization increases and space becomes a premium, indoor hydroponic farming is not just a choice; itâ€™s a necessity. And for such an important mission, of building an equitable food system, we need solutions that are smart, intuitive, and efficient.\n\nA Global Problem\n\nFacing the Future of Food: A Critical Juncture\n\nDemand Skyrockets, Supply Dwindles\n\nBy the year 2050, our world will be home to nearly 10 billion people, all needing sustenance. Yet, the capacity of our planet to provide is under unprecedented pressure. Valuable agricultural land is vanishing, swallowed by urban expansion, challenged by changing climates, and degraded by practices that fail to stand the test of sustainability.\n\nThe Cost of Conventional Farming')

In [10]:
text[1]

Document(page_content='The Cost of Conventional Farming\n\nThe way we grow food now demands too much water and leans heavily on chemicals, making it untenable. Our current methods deplete vital water resources and depend on pesticides that inflict damage on the environment, signaling a need for change.\n\nLocalized Production, Global Vulnerability\n\nA glaring example of our food systemâ€™s fragility is seen in the United States, where over 90% of its leafy greens are produced in just two statesâ€”California and Arizona. This concentration not only poses a risk to US food security but also highlights a global challenge. Countries, especially those in the Middle East with a heavy reliance on food imports, find themselves at risk of supply disruptions. Such dependency on a few production hubs for essential foods increases the likelihood of shortages, threatening food security worldwide.\n\nThe Solution:\n\nSustainable Food Security\n\nThrough Indoor Vertical Farming')

<font color="green">Splitting with metadata</font>

In [11]:
metadatas = [{"chunk": 0}, {"chuck": 1}]

document = text_splitter.create_documents(
    [loaded_data[0].page_content, loaded_data[0].page_content],
    metadatas=metadatas
)

In [12]:
document[0]

Document(metadata={'chunk': 0}, page_content='What is our goal?\n\nEnsure that fresh, nutritious, and\nsustainable food is accessible to all.\n\nAs urbanization increases and space becomes a premium, indoor hydroponic farming is not just a choice; itâ€™s a necessity. And for such an important mission, of building an equitable food system, we need solutions that are smart, intuitive, and efficient.\n\nA Global Problem\n\nFacing the Future of Food: A Critical Juncture\n\nDemand Skyrockets, Supply Dwindles\n\nBy the year 2050, our world will be home to nearly 10 billion people, all needing sustenance. Yet, the capacity of our planet to provide is under unprecedented pressure. Valuable agricultural land is vanishing, swallowed by urban expansion, challenged by changing climates, and degraded by practices that fail to stand the test of sustainability.\n\nThe Cost of Conventional Farming')

In [13]:
print(document[0])

page_content='What is our goal?

Ensure that fresh, nutritious, and
sustainable food is accessible to all.

As urbanization increases and space becomes a premium, indoor hydroponic farming is not just a choice; itâ€™s a necessity. And for such an important mission, of building an equitable food system, we need solutions that are smart, intuitive, and efficient.

A Global Problem

Facing the Future of Food: A Critical Juncture

Demand Skyrockets, Supply Dwindles

By the year 2050, our world will be home to nearly 10 billion people, all needing sustenance. Yet, the capacity of our planet to provide is under unprecedented pressure. Valuable agricultural land is vanishing, swallowed by urban expansion, challenged by changing climates, and degraded by practices that fail to stand the test of sustainability.

The Cost of Conventional Farming' metadata={'chunk': 0}


<font color="green">Embedding: transform chunks of text in chunks of numbers</font>

- Vector databases work very fast with numbers.
- See the documentation page <a href="">here.</a>
- See the list of embedding model providers <a href="">here.</a>
- The base Embeddings class in LangChain provides two methods: one for **embedding documents** and one for **embeddng a query.**

In [20]:
from langchain_community.embeddings import LlamafileEmbeddings

embeddings_model = LlamafileEmbeddings()

In [21]:
chunks_of_text = [
 'What is our goal?',
 'Ensure that fresh,',
 'nutritious, and',
 'sustainable food is',
 'is accessible to all.',
 'As urbanization increases',
 'and space becomes a',
 'a premium, indoor',
 'hydroponic farming is not',
 'not just a choice; itâ€™s',
 'a necessity. And for such',
 'an important mission, of',
 'of building an equitable',
 'food system, we need',
 'solutions that are smart,',
 'intuitive, and efficient.',
 'A Global Problem',
 'Facing the Future of',
 'of Food: A Critical',
 'Juncture',
 'Demand Skyrockets, Supply',
 'Dwindles',
 'By the year 2050, our',
 'our world will be home to',
 'to nearly 10 billion',
 'people, all needing',
 'sustenance. Yet, the',
 'the capacity of our',
 'our planet to provide is',
 'is under unprecedented',
 'pressure. Valuable',
 'agricultural land is',
 'is vanishing, swallowed',
 'by urban expansion,',
 'challenged by changing',
 'climates, and degraded by',
 'by practices that fail to',
 'to stand the test of',
 'of sustainability.',
 'The Cost of Conventional',
 'Farming',
 'The way we grow food now',
 'now demands too much',
 'water and leans heavily',
 'on chemicals, making it',
 'it untenable. Our current',
 'methods deplete vital',
 'water resources and',
 'and depend on pesticides',
 'that inflict damage on',
 'on the environment,',
 'signaling a need for',
 'for change.',
 'Localized Production,',
 'Global Vulnerability',
 'A glaring example of our',
 'our food systemâ€™s',
 'fragility is seen in the',
 'the United States, where',
 'over 90% of its leafy',
 'greens are produced in',
 'in just two',
 'two statesâ€”California',
 'and Arizona. This',
 'concentration not only',
 'poses a risk to US food',
 'security but also',
 'highlights a global',
 'challenge. Countries,',
 'especially those in the',
 'the Middle East with a',
 'a heavy reliance on food',
 'imports, find themselves',
 'at risk of supply',
 'disruptions. Such',
 'dependency on a few',
 'few production hubs for',
 'for essential foods',
 'increases the likelihood',
 'of shortages, threatening',
 'food security worldwide.',
 'The Solution:',
 'Sustainable Food Security',
 'Through Indoor Vertical',
 'Farming',
 'We are committed to',
 'to redefining how food is',
 'is grown. Indoor vertical',
 'farming stands as a',
 'a beacon of sustainable',
 'food security, offering a',
 'a resilient and',
 'and environmentally',
 'friendly alternative to',
 'to traditional',
 'agriculture. Join us in',
 'in embracing a future',
 'where food is not only',
 'abundant but also',
 'sustainable for our',
 'our planet.',
 'AGEYEâ€™s Mission aligns',
 'with UN Sustainable',
 'Development Goals',
 'What is hydroponic',
 'vertical farming?',
 'Hydroponic vertical',
 'farming is a modern',
 'technique of growing',
 'plants in stacked layers',
 'without soil, using',
 'nutrient-rich water. This',
 'method allows for',
 'for efficient use of',
 'of space, faster plant',
 'growth, and',
 'and pesticide-free',
 'produce, making it a',
 'a sustainable option for',
 'for urban farming.',
 'Basic Components',
 'Growing Medium:',
 'Instead of soil,',
 'materials like perlite,',
 'coconut coir, or rock',
 'wool support plant roots.',
 'Nutrient Solution:',
 'A water-based solution',
 'enriched with essential',
 'minerals and nutrients.',
 'Water Delivery System:',
 'Pumps and tubing deliver',
 'the nutrient solution to',
 'to plant roots.',
 'Lighting:',
 'LED grow lights mimic',
 'natural sunlight,',
 'providing the necessary',
 'spectrum for',
 'for photosynthesis.',
 'Environmental Control:',
 'Systems to regulate',
 'temperature, humidity,',
 'and CO2 levels, ensuring',
 'optimal growth',
 'conditions.',
 'Vertical Racks',
 'The plants are cultivated',
 'in vertical racks within',
 'a controlled environment.',
 'Each layer has its own',
 'own lighting and nutrient',
 'delivery system, ensuring',
 'uniform growth. These',
 'racks allow for more',
 'plants to be grown in the',
 'the same space, making it',
 'it ideal for urban',
 'environments or areas',
 'with limited space.',
 'Vertical farming',
 'maximizes space',
 'efficiency by stacking',
 'layers of plants.',
 'Controlled Environment',
 'Hydroponic indoor',
 'vertical farms operate in',
 'in a controlled',
 'environment where factors',
 'such as temperature,',
 'humidity, light, and CO2',
 'CO2 levels are',
 'are meticulously managed.',
 'This control minimizes',
 'the risk of pests and',
 'and diseases, reduces the',
 'the need for pesticides,',
 'and creates a stable',
 'growing environment that',
 'enhances plant health and',
 'and productivity.',
 'Ebb and Flow (Flood &',
 '& Drain) System',
 'The Ebb and Flow system',
 'is one of the most',
 'popular hydroponic',
 'systems due to its',
 'its simplicity and',
 'and efficiency. The',
 'The growing area is',
 'is periodically flooded',
 'with the nutrient',
 'solution, submerging the',
 'the plant roots. After a',
 'a set period, the',
 'the nutrient solution is',
 'is drained back into the',
 'the reservoir, allowing',
 'the roots to receive',
 'oxygen. This process',
 'repeats at regular',
 'intervals, ensuring that',
 'the plants receive the',
 'the nutrients and oxygen',
 'they need.',
 'Starting Your Own Indoor',
 'Vertical Farm',
 'Initial Consideration',
 'Space Requirements:',
 'Determine the size and',
 'and layout of your',
 'growing area.',
 'Crop Selection:',
 'Choose crops that are',
 'are well-suited for',
 'for hydroponic systems,',
 'such as leafy greens,',
 'herbs, and certain',
 'fruits.',
 'Setting Up',
 'Infrastructure:',
 'Build or purchase',
 'vertical racks and',
 'and install the',
 'the hydroponic system.',
 'Lighting:',
 'Set up LED grow lights to',
 'to ensure proper light',
 'coverage.',
 'Nutrient Solution:',
 'Prepare and maintain the',
 'the nutrient solution',
 'according to the needs of',
 'of your chosen crops.',
 'Monitoring:',
 'Regularly check pH',
 'pH levels, nutrient',
 'concentration, and',
 'and environmental',
 'conditions with Digital',
 'Cultivation powered by',
 'by AGEYE.',
 'Automation:',
 'Implement automated',
 'systems for irrigation,',
 'lighting, climate control',
 'and plant movement to',
 'to optimize growing',
 'conditions and reduce',
 'manual labor.',
 'Maintenance and Care',
 'Daily Checks:',
 'Inspect plants, and',
 'and monitor system',
 'performance.',
 'Nutrient Management:',
 'Adjust nutrient solutions',
 'as plants grow.',
 'Harvesting:',
 'Plan for regular',
 'harvesting to maintain',
 'continuous production.',
 'Differences Between',
 'Hydroponic and',
 'and Conventional Growing',
 'The global population is',
 'is growing by 1% every',
 'year and is projected to',
 'to reach 9.8 billion by',
 'by 2050. Vertical farming',
 'could provide a solution',
 'to cultivating enough',
 'food. Vertical farms can',
 'can be built in many',
 'unconventional indoor',
 'spaces, including',
 'warehouses, skyscrapers,',
 'shipping containers, and',
 'and old industrial',
 'buildings and factories.',
 'Controlled Environment',
 'Agriculture (CEA)',
 'regulates temperature,',
 'humidity, lighting,',
 'water, nutrition, and',
 'and even carbon dioxide',
 'to create a perfect',
 'indoor microclimate for',
 'for growing.',
 'Conventional Farming',
 'Hydroponic Farming',
 'Soil vs. Soilless',
 'Relies on natural soil,',
 'which can be inconsistent',
 'in nutrient content and',
 'and subject to erosion',
 'and degradation.',
 'Uses a water-based',
 'nutrient solution,',
 'providing consistent and',
 'and optimal nutrient',
 'delivery.',
 'Water Usage',
 'Requires large amounts of',
 'of water, much of which',
 'is lost to evaporation',
 'and runoff.',
 'Recirculates water,',
 'drastically reducing',
 'water waste. Uses 95%',
 '95% less water.',
 'Space Utilization',
 'Typically requires large',
 'expanses of land.',
 'Maximizes space by',
 'by stacking layers of',
 'of plants vertically.',
 'Uses 99% less land.',
 'Environmental Impact',
 'Can contribute to soil',
 'depletion, water',
 'pollution, and high',
 'pesticide use.',
 'Minimizes environmental',
 'impact through efficient',
 'resource use and reduced',
 'need for chemical inputs.',
 'Organic Potential',
 'Often relies on',
 'on pesticides,',
 'herbicides, and synthetic',
 'fertilizers to protect',
 'crops and enhance growth.',
 'Vertical farms are often',
 '100% organic, using no',
 'no pesticides,',
 'herbicides, or synthetic',
 'fertilizers.',
 'AGEYE\nGrow',
 'Everything You Need for',
 'for Automated Vertical',
 'Farming',
 'automated vertical',
 'farming icon',
 'Scalable Vertical Racks',
 '(4-20 Levels)',
 'Engineered to maximize',
 'both space efficiency and',
 'and plant production.',
 'automated vertical',
 'farming icon',
 'Full-Spectrum LED Grow',
 'Lights',
 'Mimics natural light,',
 'enhancing plant growth.',
 'automated vertical',
 'farming icon',
 'Advanced Hydroponic',
 'Fertigation System',
 'Delivers precise',
 'nutrients and water for',
 'for optimal growth.',
 'automated vertical',
 'farming icon',
 'Durable Food-Grade',
 'Growing Trays',
 'These trays ensure a safe',
 'and hygienic environment',
 'for edible crops.',
 'automated vertical',
 'farming icon',
 'Seeding & Transplanting',
 'Solutions',
 'Streamlines the early',
 'stages of plant growth.',
 'automated vertical',
 'farming icon',
 'Elevator & Transit Robots',
 'Automates plant movement,',
 'reducing labor.',
 'automated vertical',
 'farming icon',
 'Dedicated Germination',
 'Nurseries',
 'Optimizes conditions for',
 'for seed germination.',
 'automated vertical',
 'farming icon',
 'Harvesting & Root Cutting',
 'Technology',
 'Efficiently prepares',
 'plants for market.',
 'automated vertical',
 'farming icon',
 'Automated Tray Cleaning',
 'Maintains high sanitary',
 'standards with minimal',
 'effort & time.',
 'automated vertical',
 'farming icon',
 'Packaging & Traceability',
 'Tools',
 'Ensures product quality',
 'and traceability.',
 'automated vertical',
 'farming icon',
 'Automated Tray Storage',
 'Solutions',
 'Saves space and improves',
 'workflows.',
 'automated vertical',
 'farming icon',
 'Interlayer Horizontal',
 'Airflow',
 'Ensures uniform air',
 'air distribution for',
 'for plant health.',
 'automated vertical',
 'farming icon',
 'Precision Environment',
 'Sensors',
 'Monitors and adjusts',
 'growing conditions.',
 'automated vertical',
 'farming icon',
 'Automated Crop Scouting',
 'Powerful AI technology',
 'for early issue',
 'detection.',
 'automated vertical',
 'farming icon',
 'Farm Management Software',
 'Central control for your',
 'vertical farm operations.',
 'automated vertical',
 'farming icon',
 'Integrated Power & PLC',
 'PLC Systems',
 'Reliable and efficient',
 'energy and control',
 'systems.',
 'Grow More.\nWorry Less.',
 'Digital Cultivationâ€™s',
 'flexibility allows indoor',
 'growers with unique farm',
 'configurations to',
 'to personalize the',
 'the platform to suit',
 'their individual',
 'requirements, enhancing',
 'efficiency and increasing',
 'productivity throughout',
 'their operations.',
 'AGEYE\n\nIntelligence',
 'The leading AI-powered',
 'crop intelligence',
 'platform for controlled',
 'environment agriculture',
 '(CEA).',
 'Our unique, patented crop',
 'intelligence technology',
 'is designed from the',
 'the ground up for indoor',
 'farming and honed using',
 'millions of real farm',
 'data sets, resulting in',
 'in unparalleled',
 'efficiency and',
 'and productivity across',
 'your entire farming',
 'operation. AGEYEâ€™s AI',
 'AI platform',
 'revolutionizes indoor',
 'farming with three',
 'advanced AI models, each',
 'specifically designed to',
 'to address distinct',
 'aspects of indoor farm',
 'management and enhance',
 'operational efficiencies.',
 'LAM\n\nLarge Action Models',
 'Our LAM technology',
 'processes diverse',
 'datasets to forecast and',
 'and refine plant growth',
 'strategies, ensuring',
 'precise adjustments are',
 'are made for maximum',
 'yield, profitability, and',
 'and operational',
 'efficiency. This',
 'transforms insights into',
 'tangible actions,',
 'streamlining farm',
 'operations and enhancing',
 'productivity.',
 'LLM',
 'Large Language Models',
 'By analyzing vast amounts',
 'of textual data,',
 'including environmental',
 'conditions, input costs,',
 'and crop health reports,',
 'alongside deep',
 'horticultural knowledge,',
 'we convert complex',
 'information into',
 'actionable insights using',
 'generative AI. This',
 'empowers farmers with',
 'smarter, data-driven',
 'farming practices that',
 'optimize resources and',
 'and improve',
 'decision-making.',
 'CNN',
 'Convolutional Neural',
 'Networks',
 'Our intelligent crop',
 'scouting system utilizes',
 'visual inputs to provide',
 'actionable insights on',
 'on growth development,',
 'enabling the early',
 'detection of issues like',
 'plant stress, pathogens,',
 'or pests. This proactive',
 'monitoring saves time,',
 'reduces resource usage,',
 'and minimizes the risk of',
 'of crop loss.',
 'AGEYEâ€™s',
 'Truly Intelligent',
 'FarmingTM',
 'By optimizing crop yields',
 'and streamlining',
 'operations, our platform',
 'ensures farms operate at',
 'at peak performance,',
 'reducing costs and',
 'and boosting',
 'productivity.',
 'Innovating with',
 'Non-Destructive',
 'Data Collection and',
 'and Real-Time Analytics',
 'We are proud to introduce',
 'the industryâ€™s first',
 'non-destructive data',
 'collection and real-time',
 'distribution modeling',
 'system, tailored for',
 'for commercial growers.',
 'The system captures a',
 'a wealth of data for',
 'for every single plant,',
 'including detailed',
 'imagery and environmental',
 'readings. This',
 'revolutionary approach',
 'captures and analyzes',
 'critical metrics like',
 'fresh mass, temperature,',
 'airflow, and height',
 'distribution, empowering',
 'growers with invaluable',
 'insights.',
 'Per Plant Visibility',
 'AGEYEâ€™s Autonomous',
 'Robotic Imaging System',
 '(ARIS) redefines',
 'precision agriculture by',
 'by offering per-plant',
 'visibility. It allows you',
 'you to view distributions',
 'like fresh mass and VPD',
 'VPD across multiple',
 'levels or individual',
 'trays. This capability',
 'enables growers to',
 'to identify trends,',
 'detect issues, and take',
 'proactive steps. Whether',
 'a plant is nestled in a',
 'a corner tray or at the',
 'the top tier of a rack,',
 'it receives individual',
 'monitoring and analysis,',
 'ensuring the highest',
 'level of care and',
 'and precision.',
 'Industry Solutions',
 'Beyond Leafy Greens',
 'Vertical farms are not',
 'not only feasible and',
 'and financially',
 'successful for',
 'for cultivating leafy',
 'greens, herbs, and other',
 'high-value crops, but',
 'but they can also be key',
 'key players in the next',
 'generation of',
 'of agriculture. Vertical',
 'farms offer a broad',
 'spectrum of',
 'of possibilities, from',
 'molecular farming that',
 'generates crucial',
 'medicines and sustainable',
 'materials, to repurposing',
 'vacant office spaces, to',
 'to the efficient and safe',
 'production of cannabis',
 'and hemp.',
 'HYVE',
 'As a Biomanufacturing',
 'Platform for the',
 'Future of Food and',
 'and Pharmaceuticals',
 'Transforming food and',
 'and medicine production',
 'through advanced genetic',
 'engineering of plant',
 'cells to create',
 'therapeutic proteins,',
 'vaccines, and other',
 'valuable biological',
 'products.',
 'HYVE',
 'As a Platform for the',
 'The Future of Cannabis',
 'and Hemp',
 'Revolutionize cannabis',
 'and hemp cultivation with',
 'precise environmental',
 'control, quality',
 'optimization, and',
 'and lifecycle',
 'traceability, setting new',
 'new standards for',
 'for sustainable and',
 'and efficient production.',
 'HYVE',
 'As a Platform for',
 'Revitalizing Commercial',
 'Real Estate',
 'Our innovative solution',
 'repurposes vacant office',
 'spaces into productive',
 'vertical farms,',
 'addressing rising',
 'vacancies while meeting',
 'the demand for',
 'for sustainable food',
 'production in urban',
 'areas.',
 'Collaborate',
 'AGEYE is a strong',
 'advocate for',
 'for collaboration as a',
 'a driving force in',
 'in advancing the',
 'the Controlled',
 'Environment Agriculture',
 '(CEA) industry. If',
 'If youâ€™re an OEM,',
 'vendor, or solutions',
 'provider looking to join',
 'our ecosystem, please get',
 'get in touch',
 'AGEYE\n\nOperations',
 'An indoor farm management',
 'platform that brings',
 'together all data and',
 'and operational controls',
 'into one easy-to-use',
 'interface.',
 'What do we do?',
 'Comprehensive Farm',
 'Management and',
 'Resource Planning for CEA',
 'CEA Operations',
 'Our AI-powered farm',
 'management platform',
 'provides growers with an',
 'an all-encompassing',
 'toolkit for managing',
 'their entire farming',
 'operation. It covers',
 'everything from planting',
 'schedules and crop',
 'performance metrics to',
 'to energy consumption and',
 'and resource',
 'distribution, ensuring',
 'improved visibility and',
 'and optimized cost',
 'efficiency. This lays the',
 'the groundwork for',
 'for undeniable',
 'profitability. By',
 'By consolidating complete',
 'farm management, advanced',
 'crop intelligence, and',
 'and precise operational',
 'control into a single',
 'dashboard, our platform',
 'ensures peak performance,',
 'representing the cutting',
 'edge of intelligent farm',
 'management software.',
 'Centralized Control',
 'Accurate Forecasting',
 'Digital Twins',
 'Operational Ease',
 'Centralized Dashboard',
 'Digital Cultivation is a',
 'a centralized dashboard',
 'that gathers all data',
 'into a user-friendly',
 'platform, digitizing the',
 'the entire farming',
 'process for a',
 'a comprehensive',
 'operational overview.',
 'This centralization',
 'enables data-driven',
 'decisions, streamlines',
 'efficiency, and increases',
 'productivity. Stay',
 'updated with instant',
 'alerts and notifications.',
 'Alerts and Notifications',
 'Stay ahead of potential',
 'problems with our',
 'our advanced alerts and',
 'and notifications system.',
 'Receive immediate updates',
 'on crucial issues, from',
 'leak detection in your',
 'irrigation system to',
 'to deviations in',
 'in environmental',
 'conditions like',
 'temperature and humidity.',
 'Our platform ensures',
 'youâ€™re always the first',
 'to know when attention is',
 'is needed, allowing for',
 'for swift action to',
 'to prevent damage to your',
 'crops.',
 'Efficient Crop Planning',
 'and Scheduling',
 'With our platformâ€™s',
 'advanced algorithms,',
 'tailor your farming',
 'activities to match crop',
 'cycles perfectly,',
 'enabling precise planning',
 'and scheduling. This',
 'ensures that planting,',
 'nurturing, and harvesting',
 'are carried out at the',
 'the optimal times. This',
 'strategic planning aligns',
 'with market demand,',
 'ensuring your produce is',
 'is ready at the right',
 'moment.',
 'AI Optimized Crop',
 'Schedules',
 'Our platform uses AI to',
 'to analyze vast amounts',
 'of data, from environment',
 'conditions to plant',
 'growth rates, predicting',
 'the best planting',
 'schedules for maximum',
 'yield. This intelligent',
 'system adjusts in',
 'in real-time to changes',
 'in environmental factors',
 'or crop performance,',
 'providing dynamic',
 'scheduling that adapts to',
 'to your farmâ€™s unique',
 'conditions. By optimizing',
 'crop schedules, you',
 'you reduce waste, enhance',
 'productivity, and',
 'and significantly',
 'increase your farming',
 'operationâ€™s overall',
 'efficiency.',
 'Inventory and Supply',
 'Chain Management',
 'Manage your resources',
 'effortlessly. Track',
 'seeds, nutrients, and',
 'and other supplies with',
 'precision. Plan your',
 'harvests and distribution',
 'effectively to meet',
 'demand and reduce waste.',
 'Data Analysis and',
 'and Reporting',
 'Gain valuable insights',
 'into yield forecasts,',
 'monitor resource',
 'consumption, and assess',
 'operational efficiency to',
 'to fine-tune your farming',
 'strategies. By comparing',
 'historical growth cycles',
 'and establishing',
 'performance benchmarks,',
 'our comprehensive reports',
 'guide you in refining',
 'practices to boost your',
 'farmâ€™s output and',
 'and profitability.',
 'Energy Management',
 'Optimize your energy use.',
 'Integrate sustainable',
 'energy solutions and',
 'and monitor consumption',
 'to reduce costs and',
 'and environmental impact.',
 'Our energy management',
 'tools help you run a more',
 'efficient and green',
 'operation.',
 'Comprehensive System',
 'Control',
 'Our software places full',
 'control of your indoor',
 'farmâ€™s systems and',
 'and operational',
 'activities at your',
 'fingertips. With just a',
 'a few clicks, manage',
 'every aspect of your',
 'environment â€“ from',
 'lighting and pumps to',
 'to airflow â€“ ensuring',
 'each element operates at',
 'at peak efficiency. The',
 'The platform empowers you',
 'you with the ability to',
 'to streamline and oversee',
 'key farming tasks such as',
 'as scouting, planting,',
 'and harvesting. This',
 'holistic approach to farm',
 'management simplifies',
 'complex processes,',
 'allowing you to adjust',
 'conditions in real-time',
 'and automate routine',
 'operations for',
 'for unparalleled',
 'productivity and results.',
 'By integrating control',
 'over both hardware',
 'systems and farming',
 'activities, our software',
 'delivers a seamless,',
 'efficient farming',
 'experience that saves',
 'time and enhances yield.',
 'Unmatched Integration',
 'Designed with unlimited',
 'integration capabilities',
 'that enable seamless',
 'connections with a wide',
 'range of other systems',
 'through API integration',
 'and ensuring data',
 'interoperability. This',
 'flexibility enables farms',
 'to easily share, analyze,',
 'and manage data across',
 'different vendor',
 'technologies and',
 'and platforms, enhancing',
 'operational efficiency',
 'and decision-making',
 'processes.',
 'Food Safety and Automated',
 'Record Keeping',
 'Itâ€™s never been easier',
 'to adhere to the FDAâ€™s',
 'Produce Safety Rules by',
 'by enhancing traceability',
 'and record-keeping,',
 'crucial for managing',
 'safety incidents. Digital',
 'Cultivation automates',
 'compliance with water and',
 'and soil quality',
 'standards, ensuring',
 'environmental factors',
 'meet FDA guidelines.',
 'Additionally, our',
 'our platform facilitates',
 'the implementation of',
 'of worker health and',
 'and hygiene practices,',
 'directly contributing to',
 'to the prevention of',
 'of contamination and the',
 'the spread of foodborne',
 'illnesses. This',
 'comprehensive approach',
 'supports farms in',
 'in upholding stringent',
 'food safety standards,',
 'promoting a safer produce',
 'supply chain.',
 'Transparent Food',
 'Traceability All the Way',
 'Way to the Customer',
 'In addition to offering',
 'comprehensive control and',
 'and oversight of plant',
 'growth on our farms, we',
 'we extend this',
 'transparency to our',
 'our customers who',
 'who purchase our',
 'our farm-fresh produce.',
 'Through a simple QR code',
 'scan on the packaging,',
 'users can access a',
 'a summary of how each',
 'batch of produce was',
 'was cultivated without',
 'human touch. This',
 'commitment to',
 'to transparency ensures',
 'that consumers have a',
 'a clear understanding of',
 'of the quality and safety',
 'of their food.'
]

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

d:\active-learning\GENERATIVEAI\genai_bootcamp\nenv\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
